In [1]:
# Add autoreloatd
%reload_ext autoreload
%autoreload 2

In [15]:
from jose import jwt
from uuid import UUID
import requests
import json
import pandas as pd
import pickle
from dataclasses import asdict
from pathlib import Path
import jsonlines
from elasticsearch import Elasticsearch

from redbox.models import Settings

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

pd.set_option("display.max_colwidth", None)

ENV = Settings()
ENV.minio_host = "localhost"
ENV.elastic.host = "localhost"

☝️⚠️ _expand cell to run imports_

# Set baseline evaluation for current/deployed Redbox RAG chat endpoint  <a class="anchor" id="title"></a>

-----------

**Evaluate Redbox RAG chat on one stable, numbered version of these data**

-----------------------

## Table of Contents <a class="anchor" id="toc"></a>

* 1. Introduction
    * [Overview](#overview)
    * [Metrics](#metrics)
        - [Contextual Precision]()
        - [Contextual Recall]()
        - [Contextual Relevancy]()
        - [Fathfulness]()
        - [Answer Relevancy]()
        - [Hallucination]()
* 2. Setup
    * [Set version of the evaluation dataset](#setversion)
    * [Run Redbox locally](#run-redbox)
    * [Load embeddings into the index](#load-embeddings)
* 3. Set baseline
    * [Get files that correspond to the version of evaluation dataset](#bl-files)
    * [Load Evaluation Dataset into test cases](#bl-load-test-cases)
    * [Generate `actual_output` using RAG and evaluation dataset](#bl-evaluate)
        - [Retrieval Evaluation Metrics]()
        - [Generation Evaluation Metrics]()
    * [Analyse evaluation results](#bl-analysis)

------------

## 1. Introduction

### Overview <a class="anchor" id="overview"></a>

This notebook allows you to use the Redbox RAG chat **endpoint** to set baseline evaluation metrics for the current/deployed RAG chat endpoint.

Redbox RAG chat is made up of many components that work together to give the final RAG pipeline. Each component can be optimised to improve the overall performance of the RAG pipeline for Redbox tasks. In order to track if changes made are improving or degrading Redbox performance, we need to establish an evaluation framework. The overall RAG pipeline can be broken down into two main parts:

1. Retrieval - searching and returning the most relevant documents to answer a user question
2. Generation - the ouput of the LLM after considering the retrieved documents, any prompts provided and the user question

This notebook tests both the retrieval and generation sides of the RAG pipeline using specific metrics for each, using the `DeepEval` framework.

For consistency across the team, it is important to evaluate Redbox RAG chat on one stable, numbered version of these data.

[Back to top](#title)

### Metrics <a class="anchor" id="metrics"></a>

Retrieval metrics
- Contextual Precision
- Contextual Recall
- Contextual Relevancy

Generation metrics
- Faithfulness
- Answer Relevancy
- Hallucination


#### Contextual Precision

The contextual precision metric measures your RAG pipeline's retriever by evaluating whether nodes in your `retrieval_context` that are relevant to the given `input` are ranked higher than irrelevant ones.

#### Contextual Recall

The contextual recall metric measures the quality of your RAG pipeline's retriever by evaluating the extent of which the `retrieval_context` aligns with the `expected_output`.

#### Contextual Relevancy

The contextual relevancy metric measures the quality of your RAG pipeline's retriever by evaluating the overall relevance of the information presented in your `retrieval_context` for a given `input`.

#### Faithfulness

The faithfulness metric measures the quality of your RAG pipeline's generator by evaluating whether the `actual_output` factually aligns with the contents of your `retrieval_context`. `deepeval`'s faithfulness metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the `FaithfulnessMetric`, you need to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

#### Answer Relevancy
The answer relevancy metric measures the quality of your RAG pipeline's generator by evaluating how relevant the actual_output of your LLM application is compared to the provided `input`. `deepeval`'s answer relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the AnswerRelevancyMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`

[Back to top](#title)

#### Hallucination
The hallucination metric determines whether your LLM generates factually correct information by comparing the `actual_output` to the provided `context`.

##### Required Arguments
To use the HallucinationMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

-------------------

## 2. Setup

### Set global variables <a class="anchor" id="setversion"></a>

**Evaluate Redbox RAG chat on one stable, numbered version of these data**

**Set the version of the evaluation dataset you are using to evalute Redbox in the cell below**   <a class="anchor" id="setversion"></a>

In [6]:
DATA_VERSION = "0.1.0"

Embedding and retrieval is locked to a particular embedding model, which should be tied to a single index in the vector stoer. Here we default to the `EMBEDDING_MODEL` environment variable, which will match production if set via `.env.example`.

In [ ]:
MODEL = ENV.embedding_model
INDEX = f"{DATA_VERSION}-{MODEL}".lower()

Run the cell below to set up the required folder structure (it will not overwrite folders and files if they already exist)

In [7]:
ROOT = Path.cwd().parents[1]
EVALUATION_DIR = ROOT / "notebooks/evaluation"

V_ROOT = EVALUATION_DIR / f"data/{DATA_VERSION}"
V_RAW = V_ROOT / "raw"
V_SYNTHETIC = V_ROOT / "synthetic"
V_CHUNKS = V_ROOT / "chunks"
V_RESULTS = V_ROOT / "results"
V_EMBEDDINGS = V_ROOT / "embeddings"

V_ROOT.mkdir(parents=True, exist_ok=True)
V_RAW.mkdir(parents=True, exist_ok=True)
V_SYNTHETIC.mkdir(parents=True, exist_ok=True)
V_CHUNKS.mkdir(parents=True, exist_ok=True)
V_RESULTS.mkdir(parents=True, exist_ok=True)
V_EMBEDDINGS.mkdir(parents=True, exist_ok=True)

To save on API costs, we only need to generate a particular version of the evaluation dataset once. If you are using a previously generaterated evalutation dataset, **please download it from shared team location (Google Drive).**

It's helpful for all calls to share a dummy user. Set that here.

In [8]:
USER_UUID = UUID("aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa")

Set up the clients to connect to the backend.

In [9]:
S3_CLIENT = ENV.s3_client()
ES_CLIENT = ENV.elasticsearch_client()

INFO:root:Connecting to self managed Elasticsearch
INFO:root:Elasticsearch host = localhost


[Back to top](#title)

### Start Redbox locally <a id="run-redbox"></a>

Start docker runtime, likely with Docker Desktop. However, if you are using colima run the following terminal command

```bash
colima start --memory 8
``` 

#### First-time setup

First time users need to do the following

```bash
poetry install
```

Ensure your `.env` file has an OpenAI API key in, and has the following settings.

Note `EMBEDDING_MODEL` must also be set, and should match both production, and the embeddings shared with the dataset. If not, go back to dataset creation, and embed the documents using a model that matches production.

```bash
# === Object Storage ===

MINIO_HOST=minio
MINIO_PORT=9000
MINIO_ACCESS_KEY=minioadmin
MINIO_SECRET_KEY=minioadmin
AWS_ACCESS_KEY=minioadmin
AWS_SECRET_KEY=minioadmin

AWS_REGION=eu-west-2

# minio or s3
OBJECT_STORE=minio
BUCKET_NAME=redbox-storage-dev
```

#### Run Redbox locally

**Every time you start Redbox for evaluation (no Django frontend required), please run the following command**

```bash
make eval_backend
````

The above command will bring up everything you need for the backend (`core-api`, `worker`, `mino`, `elasticsearch` and `redis`), then create the MinIO bucket needed to store raw files

[Back to top](#title)

### Load embeddings into the index <a id="load-embeddings"></a>

When baselining, we're working with chunks in the index used by the live system, `redbox-data-chunk`.

Baselining is only meaningful if the only things in the index are the chunks from the dataset, so first we need to delete the existing data.

⚠️ THIS CODE IS DESTRUCTIVE.

In [11]:
ES_CLIENT.indices.delete(index="redbox-data-file", ignore=[400, 404])
ES_CLIENT.indices.delete(index="redbox-data-chunk", ignore=[400, 404])

/var/folders/14/6nvsrw1n2ls1xncz_bvy2x8m0000gq/T/ipykernel_94768/731841014.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  ES_CLIENT.indices.delete(index="redbox-data-file", ignore=[400, 404])
INFO:elastic_transport.transport:DELETE http://localhost:9200/redbox-data-file [status:200 duration:0.172s]
/var/folders/14/6nvsrw1n2ls1xncz_bvy2x8m0000gq/T/ipykernel_94768/731841014.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  ES_CLIENT.indices.delete(index="redbox-data-chunk", ignore=[400, 404])
INFO:elastic_transport.transport:DELETE http://localhost:9200/redbox-data-chunk [status:200 duration:0.138s]


ObjectApiResponse({'acknowledged': True})

Now we're clear to load the pre-embedded chunks in.

In [16]:
def load_chunks_from_jsonl_to_index(file_path: Path, es_client: Elasticsearch, index: str) -> None:
    with jsonlines.open(file_path, mode="r") as reader:
        for chunk_raw in reader:
            chunk = json.loads(chunk_raw)
            es_client.index(
                index=index,
                id=chunk["uuid"],
                body=chunk,
            )

In [ ]:
load_chunks_from_jsonl_to_index(
    file_path=V_EMBEDDINGS / f"{EMBEDDING_MODEL}.jsonl",
    es_client=ES_CLIENT,
    index=INDEX
)

----------

# BASELINE <a id="baseline"></a>

Run through this notebook to establish baseline evaluation metrics for the current/deployed Redbox RAG chat endpoint

------

## Set evalution baseline using current Redbox Core-API

**Use the printed out bearer token below to Authorize if you ever want to use the Swagger UI docs**

In [ ]:
bearer_token = jwt.encode({"user_uuid": str(UUID("aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa"))}, key="your-secret-key", algorithm="HS512")
print(bearer_token)

#### First need to upload files that we are going to 'RAG with'

If files have already been uploaded you can skip this step

#### Get files that correspond to the version of evaluation dataset  <a class="anchor" id="files"></a>

Copy all the files that match your {DATA_VERSION} into `notebooks/evaluation/data/{DATA_VERSION}/raw/`. Find these files on the shared Google Drive and the corresponding version number/location

**It is really important to use the same files that were used to genearte this particular version of the evaluation dataset. A mismatch between the two will result in inaccurate evaluatoin metrics**

**Only if you haven't uploaded files already** uncomment and run cell below

In [ ]:
# url = 'http://127.0.0.1:5002/file/upload'

# headers={
#     'accept': 'application/json',
#     "Authorization": f"Bearer {bearer_token}"
# }
# for file in V_RAW.glob("*.*"):
#     files = {'file': open(file, 'rb')}
#     upload_file_response = requests.post(url, headers=headers, files=files)

#     #TODO: Add some login in the loop to deal with status codes != 200
#     # if upload_file_response.status_code != 200:
#     #     print("Failed to upload data:", upload_file_response.status_code)

List files uploaded to server & view JSON response

In [ ]:
url = 'http://127.0.0.1:5002/file/'

headers={
    'accept': 'application/json',
    "Authorization": f"Bearer {bearer_token}"
}

file_list_response = requests.get(url, headers=headers)

if file_list_response.status_code == 200:
    # Parse JSON from the response
    data = file_list_response.json()
    
    # Pretty-print the JSON data
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print("Failed to retrieve data:", file_list_response.status_code)

Get file status

In [ ]:
from redbox.models import FileStatus

def pretty_upload_status(file_uuid: UUID, bearer_token: str) -> str:
    headers={
        'accept': 'application/json',
        "Authorization": f"Bearer {bearer_token}"
    }
    status = FileStatus(**requests.get(f"http://127.0.0.1:5002/file/{file_uuid}/status", headers=headers).json())

    status_title = status.processing_status.title()
    n_chunks = 0
    if status.chunk_statuses is not None:
        n_chunks = len(status.chunk_statuses)

    if status.processing_status == "embedding" and n_chunks > 0 and status.chunk_statuses is not None:
        n_chunks_embedded = len([chunk for chunk in status.chunk_statuses if chunk.embedded])
        return f"{status_title} ({n_chunks_embedded / n_chunks:.0%})"
    else:
        return status_title

statuses = [pretty_upload_status(file["uuid"], bearer_token) for file in file_list_response.json()]
statuses

-------

**Please ensure all emeddings have been completed before proceeding!**

Keep calm and go for a tea break!

--------

#### Generate `actual_output` & `retrieval_context` (using RAG chat endpoint)

In [ ]:
df = pd.read_csv(f'{V_SYNTHETIC}/ragas_synthetic_data.csv')
inputs = df['input'].tolist()

##### Using the RAG endpoint

In [ ]:
df_endpoint = df.copy()

retrieval_context = []
actual_output = []

headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer ' + bearer_token,
    'Content-Type': 'application/json',
}

url = 'http://127.0.0.1:5002/chat/rag'

for question in inputs:
    data = {
        "message_history": [
            {
                "role": "user",
                "text": question
            }
        ]
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    data = response.json()

    retrieval_context.append(data['source_documents'])
    actual_output.append(data['output_text'])

df_endpoint['actual_output'] = actual_output
df_endpoint['retrieval_context'] = retrieval_context

You can keep an eye on progress by inspecting the core API's logs using `docker compose logs -f core-api`.

#### Confirm actual_output & retrieved_context added to the dataframe

In [ ]:
df_endpoint.head()

#### Remove rows containing NaN to prevent Pydantic validation errors

In [ ]:
df_clean = df_endpoint.dropna()
df_clean.to_csv(f'{V_SYNTHETIC}/current_baseline_complete_endpoint_ragas_synthetic_data.csv', index=False)

[Back to top](#title)

----------

## Load Evaluation Dataset into test cases <a class="anchor" id="load-test-cases"></a>

Put the CSV file that you want to use for evaluation into `/notebooks/evaluation/data/synthetic_data/` directory

Import test cases from CSV

In [ ]:
from deepeval.dataset import EvaluationDataset

dataset_baseline = EvaluationDataset()
dataset_baseline.add_test_cases_from_csv_file(
    file_path=f'{V_SYNTHETIC}/current_baseline_complete_endpoint_ragas_synthetic_data.csv', # endpoint
    input_col_name="input",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    context_col_delimiter= ";",
    retrieval_context_col_name="retrieval_context",
    retrieval_context_col_delimiter= ";"
)

[Back to top](#title)

-------

## Evaluate RAG pipeline <a id="evaluate"></a>

DeepEval imports

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
)

Instantiate retrieval and generation evaluation metrics

In [ ]:
# Instantiate retrieval metrics
contextual_precision = ContextualPrecisionMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

contextual_recall = ContextualRecallMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

In [ ]:
# Instantiate generation metrics
answer_relevancy = AnswerRelevancyMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

faithfulness = FaithfulnessMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

hallucination = HallucinationMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

#### View test cases

In [ ]:
dataset_baseline.test_cases

#### Baseline Evaluation

In [ ]:
baseline_eval_results = evaluate(
    test_cases=dataset_baseline,
    metrics=[
        contextual_precision,
        contextual_recall,
        contextual_relevancy,
        answer_relevancy,
        faithfulness,
        hallucination
    ]
)

#### Save baseline evaluation results

In [ ]:
with open(f"{V_RESULTS}/current_baseline_eval_results_v{DATA_VERSION}", "wb") as f:
    pickle.dump(baseline_eval_results, f)

In [ ]:
with open(f"{V_RESULTS}/current_baseline_eval_results_v{DATA_VERSION}", "rb") as f:
    baseline_eval_results = pickle.load(f)

-------

## Analyse baseline evaluation results <a id="analysis"></a>

In [ ]:
metric_type = {
    "metric_name": ["Contextual Precision", "Contextual Recall", "Contextual Relevancy", "Answer Relevancy", "Faithfulness", "Hallucination"],
    "metric_type": ["retrieval", "retrieval", "retrieval", "generation", "generation", "generation"]
}

baseline = (
    pd.DataFrame.from_records(
        asdict(result) for result in baseline_eval_results
    )
    .explode("metrics")
    .reset_index(drop=True)
    .assign(
        metric_name = lambda df: df.metrics.apply(getattr, args=["__name__"]),
        score = lambda df: df.metrics.apply(getattr, args=["score"]),
        reason = lambda df: df.metrics.apply(getattr, args=["reason"])
    )
    .merge(
        pd.DataFrame(metric_type),
        on="metric_name"
    )
    .drop(columns=["success"])
)

baseline.to_csv(f"{V_RESULTS}/baseline.csv", index=False)
baseline.head()

#### Baseline aggregated view for current Redbox RAG chat core API endpoint

In [ ]:
(
    baseline
    .groupby(["metric_name", "metric_type"])
    .mean("score")
)

[Back to top](#title)

-----------